In [2]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

In [3]:
os.listdir('../')

['.venv',
 '5.1',
 'route.pdf',
 '.gitignore',
 'headers',
 'LICENSE',
 'README.md',
 '.git',
 '5.5',
 '1.3',
 'csv_to_tex.py',
 '.vscode',
 'pdf_to_booklet.py']